In [1]:
import numpy as np
import pandas as pd
import re

In [54]:
def arcs_to_vector(expression: re.Pattern, lines: list) -> pd.DataFrame:
    filtered = list(filter(expression.match, lines))
    vector = map(lambda x: x.split()[1:], filtered)
    colnames = ("tail", "head", "low", "cap", "cost")
    return pd.DataFrame.from_records(list(vector), columns=colnames).astype(int)

def special_nodes_to_arc(nodes: pd.DataFrame, value: int, origin=True):
    if origin:
        data = {
            "head": nodes["node"],
            "cap": nodes["flow"],
            "low": nodes["flow"],
        }
        data["tail"] = value
    else:
        data = {
            "tail": nodes["node"],
            "cap": -nodes["flow"],
            "low": -nodes["flow"],
        }
        data["head"] = value
    data["cost"] = 0
    return pd.DataFrame(data, columns=("tail", "head", "low", "cap", "cost"))

def nodes_to_vector(expression: re.Pattern, lines: list) -> pd.DataFrame:
    filtered = list(filter(expression.match, lines))
    vector = map(lambda x: x.split()[1:], filtered)
    nodes = pd.DataFrame.from_records(list(vector), columns=("node", "flow")).astype(int)
    
    source_nodes = nodes["flow"] > 0
    sink_nodes = nodes["flow"] < 0
    
    return pd.concat([
        special_nodes_to_arc(nodes[source_nodes], 25001),
        special_nodes_to_arc(nodes[sink_nodes], 25002, origin=False),
    ])


def read_file(path_to_file):
    r_arcs = re.compile("^a")
    r_nodes = re.compile("^n")
    with open(path_to_file) as f:
        content = f.readlines()
        arcs =  arcs_to_vector(r_arcs, content)
        nodes = nodes_to_vector(r_nodes, content)
    return pd.concat([arcs, nodes])

In [55]:
a = read_file("../netg/stndrd1.net")

In [56]:
a

,tail,head,low,cap,cost
0,1,102,0,100000,8550
1,1,171,0,100000,7415
2,1,101,0,100000,7178
3,1,165,0,100000,4494
4,1,151,0,100000,6289
...,...,...,...,...,...
195,196,25002,96,96,0
196,197,25002,755,755,0
197,198,25002,1577,1577,0
198,199,25002,131,131,0
